<a href="https://colab.research.google.com/github/skytecat/Cats-Dogs/blob/main/cat_and_dogs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("bhavikjikadara/dog-and-cat-classification-dataset")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/dog-and-cat-classification-dataset


In [16]:
def check_images_shapes(folder, img_size=(64, 64)):
    shapes = {}
    for filename in os.listdir(folder):
      if filename.endswith('.jpg'):
        img = Image.open(os.path.join(folder, filename))
        img = img.resize(img_size)
        img_array = np.array(img)

        shape = np.array(img).shape
        if shape not in shapes:
          shapes[shape] = 1
        else:
          shapes[shape] += 1

      return shapes

print("Форматы кошачьих изображений:")
print(check_images_shapes('/kaggle/input/dog-and-cat-classification-dataset/PetImages/Cat'))

print("\nФорматы собачьих изображений:")
print(check_images_shapes('/kaggle/input/dog-and-cat-classification-dataset/PetImages/Dog'))

Форматы кошачьих изображений:
{(64, 64, 3): 1}

Форматы собачьих изображений:
{(64, 64, 3): 1}


In [17]:
import pandas as pd
import os
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split

def load_images(folder, label, img_size=(64, 64)):
    images = []
    labels = []
    for filename in os.listdir(folder):
      if filename.endswith('.jpg'):
        try:
            img = Image.open(os.path.join(folder, filename))
            img = img.resize(img_size)
            img_array = np.array(img) / 255.0  # ← ВАЖНО: нормализация здесь!

            # if len(img_array.shape) == 2:
            #     img_array = np.stack([img_array]*3, axis=-1)
            # elif img_array.shape[2] == 4:
            #     # RGBA -> RGB: убираем альфа-канал
            #      img_array = img_array[:,:,:3]

            images.append(img_array)
            labels.append(label)
        except Exception as e:
            print(f"Ошибка: {filename} - {e}")
    return images, labels

cat_images, cat_labels = load_images('/kaggle/input/dog-and-cat-classification-dataset/PetImages/Cat', 0)
dog_images, dog_labels = load_images('/kaggle/input/dog-and-cat-classification-dataset/PetImages/Dog', 1)

/usr/local/lib/python3.12/dist-packages/PIL/TiffImagePlugin.py:950: UserWarning: Truncated File Read
  warnings.warn(str(msg))


In [3]:
X = np.array(cat_images + dog_images)
y = np.array(cat_labels + dog_labels)

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [5]:
print("Формат изображений:", X_train.shape)
print("Диапазон значений: от", X_train.min(), "до", X_train.max())

Формат изображений: (19998, 64, 64, 3)
Диапазон значений: от 0.0 до 1.0


In [6]:
!pip install tensorflow

In [7]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping

# Создаем правильную архитектуру
model = models.Sequential([
    layers.Input(shape=(64,64,3)),

    layers.Conv2D(32, (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),
    layers.Dropout(0.25),

    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),
    layers.Dropout(0.25),

    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),
    layers.Dropout(0.25),

    layers.Flatten(),
    layers.Dense(32, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')
])

# Компилируем
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# Early stopping
early_stopping = EarlyStopping(
    monitor='val_loss',  # Теперь следим за loss
    patience=5,
    restore_best_weights=True
)

# Обучаем
history = model.fit(
    X_train, y_train,
    epochs=30,
    validation_data=(X_test, y_test),
    callbacks=[early_stopping]
)

Epoch 1/30
625/625 ━━━━━━━━━━━━━━━━━━━━ 132s 208ms/step - accuracy: 0.5184 - loss: 0.6917 - val_accuracy: 0.6670 - val_loss: 0.6377
Epoch 2/30
625/625 ━━━━━━━━━━━━━━━━━━━━ 127s 204ms/step - accuracy: 0.6728 - loss: 0.6139 - val_accuracy: 0.7374 - val_loss: 0.5286
Epoch 3/30
625/625 ━━━━━━━━━━━━━━━━━━━━ 141s 203ms/step - accuracy: 0.7252 - loss: 0.5490 - val_accuracy: 0.7536 - val_loss: 0.4943
Epoch 4/30
625/625 ━━━━━━━━━━━━━━━━━━━━ 131s 209ms/step - accuracy: 0.7606 - loss: 0.4981 - val_accuracy: 0.7846 - val_loss: 0.4578
Epoch 5/30
625/625 ━━━━━━━━━━━━━━━━━━━━ 140s 205ms/step - accuracy: 0.7902 - loss: 0.4585 - val_accuracy: 0.8042 - val_loss: 0.4254
Epoch 6/30
625/625 ━━━━━━━━━━━━━━━━━━━━ 128s 205ms/step - accuracy: 0.8003 - loss: 0.4356 - val_accuracy: 0.8166 - val_loss: 0.4039
Epoch 7/30
625/625 ━━━━━━━━━━━━━━━━━━━━ 127s 204ms/step - accuracy: 0.8176 - loss: 0.4038 - val_accuracy: 0.8156 - val_loss: 0.3957
Epoch 8/30
625/625 ━━━━━━━━━━━━━━━━━━━━ 144s 208ms/step - accuracy: 0.8243 -

KeyboardInterrupt: 

In [8]:
loss, accuracy = model.evaluate(X_test, y_test)

157/157 ━━━━━━━━━━━━━━━━━━━━ 8s 51ms/step - accuracy: 0.8434 - loss: 0.3617
